In [ ]:
import numpy as np
from scipy.special import gamma, gammainc

def corrected_temperature(t_e_fpi, n_e_fpi, n_e_fit, e_min=3.0):
    """
    Estimate the true temperature assuming a Maxwellian distribution
    and a known energy cutoff (E_min).

    Parameters
    ----------
    t_e_fpi : float
        Measured temperature from FPI [eV]
    n_e_fpi : float
        Measured density from FPI [cm^-3]
    n_e_fit : float
        True density (from fit) [cm^-3]
    e_min : float
        Energy cutoff [eV] below which FPI misses data

    Returns
    -------
    t_e_true : float
        Estimated true temperature [eV]
    """
    # Define dimensionless lower limit for the integrals
    # Use normalized E / T as variable in integral, so a = E_min / T
    # We solve this iteratively since a = E_min / T_true

    # Gamma values for complete integrals (true moments)
    gamma_3_2 = gamma(1.5)
    gamma_5_2 = gamma(2.5)

    # Invert the cutoff to find T_true iteratively
    def objective(T):
        a = e_min / T
        n_ratio_model = gammainc(1.5, a)  # FPI/true
        T_ratio_model = gammainc(2.5, a) / gammainc(1.5, a)
        n_ratio_meas = n_e_fpi / n_e_fit
        T_true_est = t_e_fpi * (gamma_5_2 / gamma_3_2) / T_ratio_model
        return T - T_true_est  # Solve T = T_true_est

    from scipy.optimize import root_scalar
    sol = root_scalar(objective, bracket=[t_e_fpi, 100*t_e_fpi], method='brentq')
    
    if sol.converged:
        return sol.root
    else:
        raise RuntimeError("Failed to converge on T_true")

# Example usage
t_e_fpi = 10.0     # eV
n_e_fpi = 5.0      # cm^-3
n_e_fit = 8.0      # cm^-3

t_e_true = corrected_temperature(t_e_fpi, n_e_fpi, n_e_fit)
print(f"Corrected temperature: {t_e_true:.2f} eV")
